In [1]:
from selenium import webdriver
from selenium.common import NoSuchElementException
from selenium.webdriver.common.by import By
from selenium.webdriver.edge.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.edge.options import Options
from webdriver_manager.microsoft import EdgeChromiumDriverManager  # To download Edge WebDriver
from selenium.common.exceptions import NoSuchElementException
from itertools import chain
import pandas as pd
import os
import time
import numpy as np

In [20]:
# Set up the Edge WebDriver
options = webdriver.EdgeOptions()
options.add_argument("--disable-extensions")
options.add_argument("--no-sandbox")
options.add_argument("--headless")
service = Service(EdgeChromiumDriverManager().install())
driver = webdriver.Edge(service=service,options = options)  # Use the Edge WebDriver
# click on year to get table function, get url of each year
ori_url = "https://www.boxofficemojo.com/year/world/?ref_=bo_nb_di_tab"
def getURL(ori_url):
    driver.get(ori_url)
    # driver.implicitly_wait(5)
    time.sleep(2)
    open_list = driver.find_element(By.XPATH, '//*[@id="a-autoid-0-announce"]')
    open_list.click()
    list_year_tag = driver.find_element(By.XPATH, '//*[@id="a-popover-1"]')
    driver.execute_script("arguments[0].click();", list_year_tag)
    listYear = driver.find_element(By.XPATH, '//*[@id="a-popover-1"]/div/div/ul')
    list = listYear.find_elements(By.TAG_NAME, "li")
    dict = {}
    for li in list:
        links = li.find_elements(By.TAG_NAME, "a")
        for link in links:
            urlTemp = 'https://www.boxofficemojo.com/year/world/'
            dict.update({li.text: urlTemp + li.text})
    driver.quit()
    return dict   
listOfYear = getURL(ori_url)

In [24]:
listOfYear = dict(list(listOfYear.items())[0:25])

In [25]:
listOfYear

{'2024': 'https://www.boxofficemojo.com/year/world/2024',
 '2023': 'https://www.boxofficemojo.com/year/world/2023',
 '2022': 'https://www.boxofficemojo.com/year/world/2022',
 '2021': 'https://www.boxofficemojo.com/year/world/2021',
 '2020': 'https://www.boxofficemojo.com/year/world/2020',
 '2019': 'https://www.boxofficemojo.com/year/world/2019',
 '2018': 'https://www.boxofficemojo.com/year/world/2018',
 '2017': 'https://www.boxofficemojo.com/year/world/2017',
 '2016': 'https://www.boxofficemojo.com/year/world/2016',
 '2015': 'https://www.boxofficemojo.com/year/world/2015',
 '2014': 'https://www.boxofficemojo.com/year/world/2014',
 '2013': 'https://www.boxofficemojo.com/year/world/2013',
 '2012': 'https://www.boxofficemojo.com/year/world/2012',
 '2011': 'https://www.boxofficemojo.com/year/world/2011',
 '2010': 'https://www.boxofficemojo.com/year/world/2010',
 '2009': 'https://www.boxofficemojo.com/year/world/2009',
 '2008': 'https://www.boxofficemojo.com/year/world/2008',
 '2007': 'http

In [28]:
next_list = dict(list(listOfYear.items())[10:24])

In [29]:
next_list

{'2014': 'https://www.boxofficemojo.com/year/world/2014',
 '2013': 'https://www.boxofficemojo.com/year/world/2013',
 '2012': 'https://www.boxofficemojo.com/year/world/2012',
 '2011': 'https://www.boxofficemojo.com/year/world/2011',
 '2010': 'https://www.boxofficemojo.com/year/world/2010',
 '2009': 'https://www.boxofficemojo.com/year/world/2009',
 '2008': 'https://www.boxofficemojo.com/year/world/2008',
 '2007': 'https://www.boxofficemojo.com/year/world/2007',
 '2006': 'https://www.boxofficemojo.com/year/world/2006',
 '2005': 'https://www.boxofficemojo.com/year/world/2005',
 '2004': 'https://www.boxofficemojo.com/year/world/2004',
 '2003': 'https://www.boxofficemojo.com/year/world/2003',
 '2002': 'https://www.boxofficemojo.com/year/world/2002',
 '2001': 'https://www.boxofficemojo.com/year/world/2001'}

In [30]:
# get table in each year
def getTable(url, csv_file_path):
    service = Service(EdgeChromiumDriverManager().install())
    edge_options = Options()
    edge_options.add_argument('--disable-blink-features=AutomationControlled')
    edge_options.add_argument("--disable-extensions")
    edge_options.add_argument("--no-sandbox")
    edge_options.add_argument("--headless")
    driver = webdriver.Edge(service=service, options=edge_options)  # Use the Edge WebDriver
    driver.get(url)
    driver.sleep(2)
    year = driver.find_element(By.XPATH, '//*[@id="a-autoid-0-announce"]/span')
    tbody = driver.find_element(By.XPATH, '//*[@id="table"]/div/table[2]/tbody')
    raw_data = []

    for tr in tbody.find_elements(By.XPATH, '//tr'):
        row = [item.text for item in tr.find_elements(By.XPATH, ".//td")]
        raw_data.append(row)
    data = pd.DataFrame(raw_data)
    # drop 2 first row 
    data.drop([0, 1], inplace=True)
    # remove index
    data.reset_index(drop=True, inplace=True)
    # rename column
    data.columns = ['Rank' ,'Release Group', 'Worldwide', 'Domestic', '%', 'Foreign', '%']
    # add column year
    data['Year'] = year.text
    # convert from string to int 
    data['Year'] = data['Year'].astype(int)
    # convert from dataframe to csv file
    data.to_csv(csv_file_path, mode='a', index=False, sep=',', encoding='utf-8')
    driver.quit() # close browser
    return data
# testData = getTable(ori_url, file_path)

In [33]:
current_directory = os.getcwd() # get current directory //D:\VSC\selenium
ori_url = "https://www.boxofficemojo.com/year/world/?ref_=bo_nb_di_tab"
file_path = os.path.join(current_directory, 'data.csv')
# testData = getTable(listOfYear['2022'], file_path)

In [12]:
testData

,Rank,Release Group,Worldwide,Domestic,%,Foreign,%,Year
0,1,Avatar: The Way of Water,"$2,320,250,281","$684,075,767",29.5%,"$1,636,174,514",70.5%,2022
1,2,Top Gun: Maverick,"$1,495,696,292","$718,732,821",48.1%,"$776,963,471",51.9%,2022
2,3,Jurassic World Dominion,"$1,001,978,080","$376,851,080",37.6%,"$625,127,000",62.4%,2022
3,4,Doctor Strange in the Multiverse of Madness,"$955,775,804","$411,331,607",43%,"$544,444,197",57%,2022
4,5,Minions: The Rise of Gru,"$940,203,765","$370,270,765",39.4%,"$569,933,000",60.6%,2022
...,...,...,...,...,...,...,...,...
196,197,Listy do M. 5,"$7,328,061",-,-,"$7,328,061",100%,2022
197,198,Laid-Back Camp Movie,"$7,317,913",-,-,"$7,317,913",100%,2022
198,199,Osomatsusan the Movie 2022 Re-release,"$7,297,522",-,-,"$7,297,522",100%,2022
199,200,The New Toy,"$7,295,051",-,-,"$7,295,051",100%,2022


In [14]:
def getData_old():
    for value in listOfYear.values():
        edge_options = Options()
        edge_options.add_argument('--disable-blink-features=AutomationControlled')
        edge_options.add_argument("--disable-extensions")
        edge_options.add_argument("--no-sandbox")
        edge_options.add_argument("--headless")
        driver = webdriver.Edge(service=service, options=edge_options)  # Use the Edge WebDrivers
        getTable(value, file_path)
        driver.quit()
getData_old()

KeyboardInterrupt: 

In [34]:
def getData():
    for value in next_list.values():
        edge_options = Options()
        edge_options.add_argument('--disable-blink-features=AutomationControlled')
        edge_options.add_argument("--disable-extensions")
        edge_options.add_argument("--no-sandbox")
        edge_options.add_argument("--headless")
        driver = webdriver.Edge(service=service, options=edge_options)  # Use the Edge WebDrivers
        getTable(value, file_path)
        driver.quit()

In [ ]:
# crawl data
getData()

In [5]:
def getText(xpath):
    try:
        element = driver.find_element(By.XPATH, xpath)
        return element.text
    except NoSuchElementException:
        return np.nan

In [31]:
'''
Click Steps:
1. Click on movie's name
2. Click on Title summary
'''
temp = None
def get_info(url):
    # Setup Edge WebDriver with proper options
    edge_options = Options()
    edge_options.add_argument('--disable-blink-features=AutomationControlled')
    edge_options.add_argument("--disable-extensions")
    edge_options.add_argument("--no-sandbox")
    edge_options.add_argument("--headless")
    service = Service()  # Specify your WebDriver's path
    driver = webdriver.Edge(service=service, options=edge_options)
    driver.get(url)
    time.sleep(2)  # Increase implicit wait to handle slow loading
    overview = None
    # //*[@id="a-page"]/main/div/div[1]/div[1]/div/div/div[2]/div/span, //*[@id="a-page"]/main/div/div[1]/div[1]/div/div/div[2]/div/span, //*[@id="a-page"]/main/div/div[1]/div[1]/div/div/div[2]/p, //*[@id="a-page"]/main/div/div[1]/div[1]/div/div/div[2]/p[1]
    # //*[@id="a-page"]/main/div/div[1]/div[1]/div/div/div[2]/h1
    # //*[@id="a-page"]/main/div/div[1]/div[1]/div/div/div[2]/h1
    # //*[@id="a-page"]/main/div/div[1]/div[1]/div/div/div[2]/div/h1
    # get name
    title = driver.find_element(By.TAG_NAME,'h1')
    title = title.text
    # get overview
    try:
        overview = driver.find_element(By.XPATH, '//*[@id="a-page"]/main/div/div[1]/div[1]/div/div/div[2]/div/span')
        overview = overview.text
    except NoSuchElementException:
        try:
            overview = driver.find_element(By.XPATH, '//*[@id="a-page"]/main/div/div[1]/div[1]/div/div/div[2]/p')
            overview = overview.text 
        except NoSuchElementException:
            overview = np.nan
    # Define the expected fields and their corresponding default XPaths
    expected_fields = [
        "Domestic Distributor",
        "Domestic Opening",
        "Earliest Release Date",
        "MPAA",
        "Running Time",
        "Genres",
    ]
    
    # Initialize a dictionary to hold the retrieved data
    data_dict = {field: np.nan for field in expected_fields}

    # Function to get text from an element, returns np.nan if not found
    def get_text(xpath):
        try:
            element = driver.find_element(By.XPATH, xpath)
            return element.text
        except NoSuchElementException:
            return np.nan

    try:
        # Click on Title Summary (if needed)
        driver.find_element(By.XPATH, '//*[@id="title-summary-refiner"]/a').click()

        # Loop through the expected fields and try to find them based on order
        # The initial XPath is set to a common base, with varying indices
        for i in range(1, 7):
            field_name_xpath = f'//*[@id="a-page"]/main/div/div[3]/div[4]/div[{i}]/span[1]'
            field_value_xpath = f'//*[@id="a-page"]/main/div/div[3]/div[4]/div[{i}]/span[2]'

            field_name = get_text(field_name_xpath)
            if field_name in expected_fields:
                # If the field name matches an expected field, add its value to the dictionary
                data_dict[field_name] = get_text(field_value_xpath)
        if data_dict['Domestic Distributor'] != np.nan:
            name = data_dict['Domestic Distributor'].split('\n')
            data_dict['Domestic Distributor'] = name[0] 
        else:
            pass
    except (NoSuchElementException, AttributeError) as e:
        pass  # If there's an exception, just continue with NaN values
    # get filmmakers and cast
    try:
        driver.find_element(By.XPATH,'//*[@id="tabs"]/div/a[2]').click()
        tbody = driver.find_element(By.XPATH, '//*[@id="principalCrew"]/tbody')
        raw_data = []
        for tr in tbody.find_elements(By.XPATH, '//tr'):
            row = [item.text for item in tr.find_elements(By.XPATH, ".//td")]
            raw_data.append(row)
        raw_data.pop(0)
        empty_index = raw_data.index([])
        film_makers = raw_data[:empty_index]
        casts = raw_data[empty_index + 1:]
        films = []
        actors = []
        for film in film_makers:
            films.append(film[0])
        for cast in casts:
            actors.append(cast[0])
        driver.quit()  # Ensure the WebDriver is closed
        [data_dict[field] for field in expected_fields]
        data_dict.update({'Overview':overview})
        data_dict.update({'Film makers':films})
        data_dict.update({'Actors':actors})
    except (NoSuchElementException, AttributeError) as e:
        data_dict.update({'Overview':np.nan})
        data_dict.update({'Film makers':np.nan})
        data_dict.update({'Actors':np.nan})
    # Return the data values in the correct order
    data_dict.update({'Name': title})
    return data_dict

In [97]:
testURL = 'https://www.boxofficemojo.com/title/tt15239678/?ref_=bo_gr_ti'
testInfo = get_info(testURL)

In [98]:
print(testInfo)

{'Domestic Distributor': 'Warner Bros.', 'Domestic Opening': '$82,505,391', 'Earliest Release Date': 'February 28, 2024 (EMEA, APAC)', 'MPAA': 'PG-13', 'Running Time': '2 hr 46 min', 'Genres': 'Action Adventure Drama Sci-Fi', 'Overview': 'Paul Atreides unites with Chani and the Fremen while seeking revenge against the conspirators who destroyed his family.', 'Film makers': ['Denis Villeneuve', 'Denis Villeneuve', 'Jon Spaihts', 'Frank Herbert', 'Cale Boyter', 'Tanya Lapointe', 'Patrick McCormick', 'Mary Parent', 'Denis Villeneuve', 'Hans Zimmer', 'Greig Fraser', 'Joe Walker', 'Patrice Vermette'], 'Actors': ['Timothée Chalamet', 'Zendaya', 'Rebecca Ferguson', 'Javier Bardem'], 'Name': ['Warner Bros.', 'See full company information']}


In [54]:
again, film_test, cast_test = get_info('https://www.boxofficemojo.com/title/tt30476486/?ref_=bo_gr_ti')
print(again)
print(film_test)
print(cast_test)

{'Domestic Distributor': 'Crunchyroll', 'Domestic Opening': nan, 'Earliest Release Date': 'February 16, 2024 (Japan)', 'MPAA': nan, 'Running Time': '1 hr 25 min', 'Genres': 'Animation Comedy Drama Sport', 'Overview': 'Despite a strong field, Karasuno High volleyball team advances past preliminary round of Harutaka tournament in Miyagi prefecture to reach the third round.'}
['Susumu Mitsunaka', 'Haruichi Furudate', 'Susumu Mitsunaka', 'Miura Fumi', 'Mitsuteru Hishiyama', 'Tasuku Honda', 'Ryota Ikeda', 'Mayumi Kurashima', 'Toshihiro Maeda', 'Fumi Morihiro', 'Shôichirô Taguchi', 'Yûki Hayashi', 'Asami Tachibana', 'Yumiko Nakata', 'Junichi Uematsu']
['Ayumu Murase', 'Kaito Ishikawa', 'Yûki Kaji', 'Yûichi Nakamura']


In [96]:
again1 = get_info('https://www.boxofficemojo.com/releasegroup/gr4002566661/?ref_=bo_ydw_table_37')
again1

{'Domestic Distributor': nan,
 'Domestic Opening': nan,
 'Earliest Release Date': nan,
 'MPAA': nan,
 'Running Time': nan,
 'Genres': nan,
 'Overview': nan,
 'Film makers': nan,
 'Actors': nan,
 'Name': 'The Chosen: S4 Episodes 1-3'}

In [62]:
again2 = get_info('https://www.boxofficemojo.com/title/tt27075304/?ref_=bo_gr_ti')
again2

{'Domestic Distributor': nan,
 'Domestic Opening': nan,
 'Earliest Release Date': 'March 28, 2024 (Germany)',
 'MPAA': nan,
 'Running Time': '1 hr 58 min',
 'Genres': 'Comedy Fantasy',
 'Overview': 'Chantal will be involuntarily transported to a fairy tale world, fighting against witches and tyrant kings, but above all for her friendship with Zeynep.'}

In [36]:
def getLinks(url):
    service = Service(EdgeChromiumDriverManager().install())
    edge_options = Options()
    edge_options.add_argument('--disable-blink-features=AutomationControlled')
    edge_options.add_argument("--disable-extensions")
    edge_options.add_argument("--no-sandbox")
    edge_options.add_argument("--headless")
    driver = webdriver.Edge(service=service, options=edge_options)  # Use the Edge WebDriver
    driver.get(url)
    time.sleep(2)
    tbody = driver.find_element(By.XPATH, '//*[@id="table"]/div/table[2]/tbody')
    movies = []
    count = 2
    for tr in tbody.find_elements(By.XPATH, '//tr'):
        link = tr.find_element(By.XPATH, '//*[@id="table"]/div/table[2]/tbody/tr['+ str(count) +']/td[2]/a').get_attribute('href')
        count += 1
        if (count == 201):
            break
        #  //*[@id="table"]/div/table[2]/tbody/tr[3]/td[2]/a
        movies.append(link)
    driver.quit()
    return movies

In [37]:
getLinks('https://www.boxofficemojo.com/year/world/?ref_=bo_nb_tt_tab')

['https://www.boxofficemojo.com/releasegroup/gr2058375685/?ref_=bo_ydw_table_1',
 'https://www.boxofficemojo.com/releasegroup/gr2995671557/?ref_=bo_ydw_table_2',
 'https://www.boxofficemojo.com/releasegroup/gr361583109/?ref_=bo_ydw_table_3',
 'https://www.boxofficemojo.com/releasegroup/gr77484549/?ref_=bo_ydw_table_4',
 'https://www.boxofficemojo.com/releasegroup/gr3314176517/?ref_=bo_ydw_table_5',
 'https://www.boxofficemojo.com/releasegroup/gr4020130309/?ref_=bo_ydw_table_6',
 'https://www.boxofficemojo.com/releasegroup/gr850547205/?ref_=bo_ydw_table_7',
 'https://www.boxofficemojo.com/releasegroup/gr3197587973/?ref_=bo_ydw_table_8',
 'https://www.boxofficemojo.com/releasegroup/gr127685125/?ref_=bo_ydw_table_9',
 'https://www.boxofficemojo.com/releasegroup/gr395137541/?ref_=bo_ydw_table_10',
 'https://www.boxofficemojo.com/releasegroup/gr2055819781/?ref_=bo_ydw_table_11',
 'https://www.boxofficemojo.com/releasegroup/gr3499381253/?ref_=bo_ydw_table_12',
 'https://www.boxofficemojo.com

In [6]:
def get_all_links_old(listOfYear):
    movies = []
    for year in listOfYear.values():
        movies.append(getLinks(year))
    movies = list(np.concatenate(movies).flat)
    return movies

In [35]:
def get_all_links(next_list):
    movies = []
    for year in next_list.values():
        movies.append(getLinks(year))
    movies = list(np.concatenate(movies).flat)
    return movies

In [38]:
new_info = get_all_links(next_list)

In [ ]:
len(new_info)

1980

In [8]:
info_1 = new_info[0:350]
info_2 = new_info[350:670]
info_3 = new_info[670:990]
info_4 = new_info[990:1440]
info_5 = new_info[1440:1600]

In [9]:
info_6 = new_info[1600:1750]
info_7 = info[1750:1900]
info_8 = info[1900:]

In [86]:
info_2

['https://www.boxofficemojo.com/releasegroup/gr403788293/?ref_=bo_ydw_table_2',
 'https://www.boxofficemojo.com/releasegroup/gr1030181381/?ref_=bo_ydw_table_3',
 'https://www.boxofficemojo.com/releasegroup/gr1825526277/?ref_=bo_ydw_table_4',
 'https://www.boxofficemojo.com/releasegroup/gr3085718021/?ref_=bo_ydw_table_5',
 'https://www.boxofficemojo.com/releasegroup/gr4123480581/?ref_=bo_ydw_table_6',
 'https://www.boxofficemojo.com/releasegroup/gr2303480325/?ref_=bo_ydw_table_7',
 'https://www.boxofficemojo.com/releasegroup/gr2366394885/?ref_=bo_ydw_table_8',
 'https://www.boxofficemojo.com/releasegroup/gr2669236741/?ref_=bo_ydw_table_9',
 'https://www.boxofficemojo.com/releasegroup/gr2719109637/?ref_=bo_ydw_table_10',
 'https://www.boxofficemojo.com/releasegroup/gr423973381/?ref_=bo_ydw_table_11',
 'https://www.boxofficemojo.com/releasegroup/gr3953807877/?ref_=bo_ydw_table_12',
 'https://www.boxofficemojo.com/releasegroup/gr2718781957/?ref_=bo_ydw_table_13',
 'https://www.boxofficemoj

In [72]:
current_directory = os.getcwd()
links_path = os.path.join(current_directory, 'links.csv')
df = pd.DataFrame(info)
df.to_csv(links_path, mode='a', index=False, sep=',', encoding='utf-8')

In [10]:
service = Service(EdgeChromiumDriverManager().install())
def getMoviesInfo(links):
    movies_info = []
    for link in links:
        try:
            movies_info.append(get_info(link))
        except Exception:
            movies_info.append(np.nan)
    return movies_info

In [ ]:
movies_info_1= getMoviesInfo(info_1)

In [91]:
movies_info_2= getMoviesInfo(info_2)

NameError: name 'movies_info_1' is not defined

In [ ]:
movies_info_3= getMoviesInfo(info_3)

In [11]:
movies_info_4= getMoviesInfo(info_4)
movies_info_4=pd.to_csv('movies.csv',mode='a',index='False',sep=',', encoding='utf-8')

AttributeError: module 'pandas' has no attribute 'to_csv'

In [12]:
print(movies_info_4[0])

{'Domestic Distributor': 'Walt Disney Studios Motion Pictures', 'Domestic Opening': '$357,115,007', 'Earliest Release Date': 'April 24, 2019 (21 markets)', 'MPAA': 'PG-13', 'Running Time': '3 hr 1 min', 'Genres': nan, 'Overview': "After the devastating events of Avengers: Infinity War, the universe is in ruins. With the help of remaining allies, the Avengers assemble once more in order to reverse Thanos' actions and restore balance to the universe.", 'Film makers': ['Anthony Russo', 'Joe Russo', 'Christopher Markus', 'Stephen McFeely', 'Stan Lee', 'Jack Kirby', 'Joe Simon', 'Jack Kirby', 'Steve Englehart', 'Steve Gan', 'Bill Mantlo', 'Keith Giffen', 'Jim Starlin', 'Stan Lee', 'Larry Lieber', 'Jack Kirby', 'Steve Englehart', 'Don Heck', 'Kevin Feige', 'Alan Silvestri', 'Trent Opaloch', 'Jeffrey Ford', 'Matthew Schmidt', 'Charles Wood'], 'Actors': ['Robert Downey Jr.', 'Chris Evans', 'Mark Ruffalo', 'Chris Hemsworth'], 'Name': 'Avengers: Endgame'}


In [15]:
print(len(movies_info_4))

450


In [15]:
movies_info_5= getMoviesInfo(info_5)
print(len(movies_info_5))

160


In [11]:
movies_info_7= getMoviesInfo(info_7)
print(len(movies_info_7))

150


In [42]:
data.shape

(201, 8)

In [51]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 201 entries, 0 to 200
Data columns (total 8 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Rank           200 non-null    object
 1   Release Group  200 non-null    object
 2   Worldwide      200 non-null    object
 3   Domestic       200 non-null    object
 4   %              200 non-null    object
 5   Foreign        200 non-null    object
 6   %              200 non-null    object
 7   Year           201 non-null    int32 
dtypes: int32(1), object(7)
memory usage: 11.9+ KB


In [44]:
data

,Rank,Release Group,Worldwide,Domestic,%,Foreign,%,Year
0,1,Barbie,"$1,445,638,421","$636,238,421",44%,"$809,400,000",56%,2023
1,2,The Super Mario Bros. Movie,"$1,362,029,918","$574,934,330",42.2%,"$787,095,588",57.8%,2023
2,3,Oppenheimer,"$971,612,566","$329,862,540",34%,"$641,750,026",66%,2023
3,4,Guardians of the Galaxy Vol. 3,"$845,555,777","$358,995,815",42.5%,"$486,559,962",57.5%,2023
4,5,Fast X,"$704,875,015","$146,126,015",20.7%,"$558,749,000",79.3%,2023
...,...,...,...,...,...,...,...,...
196,197,Dream,"$8,317,251",-,-,"$8,317,251",100%,2023
197,198,Vacaciones de verano,"$8,209,854",-,-,"$8,209,854",100%,2023
198,199,On the Wandering Paths,"$8,186,415",-,-,"$8,186,415",100%,2023
199,200,Weekend Rebels,"$8,184,539",-,-,"$8,184,539",100%,2023


In [45]:
data['Year'].dtype

dtype('O')

In [46]:
data['Year'] = data['Year'].astype(str)

In [47]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 201 entries, 0 to 200
Data columns (total 8 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Rank           200 non-null    object
 1   Release Group  200 non-null    object
 2   Worldwide      200 non-null    object
 3   Domestic       200 non-null    object
 4   %              200 non-null    object
 5   Foreign        200 non-null    object
 6   %              200 non-null    object
 7   Year           201 non-null    object
dtypes: object(8)
memory usage: 12.7+ KB


In [49]:
data['Year'] = data['Year'].astype(int)

In [50]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 201 entries, 0 to 200
Data columns (total 8 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Rank           200 non-null    object
 1   Release Group  200 non-null    object
 2   Worldwide      200 non-null    object
 3   Domestic       200 non-null    object
 4   %              200 non-null    object
 5   Foreign        200 non-null    object
 6   %              200 non-null    object
 7   Year           201 non-null    int32 
dtypes: int32(1), object(7)
memory usage: 11.9+ KB
